In [1]:
from utils import FakeTRTFunc
from model import load_as_trt_model
import chess.syzygy as syzygy

use_fake = False
if use_fake:
    trt_func = FakeTRTFunc()
else:
    trt_func, _ = load_as_trt_model()
tablebase = syzygy.open_tablebase("/home/tomaz/ChessBot_v2/syzygy/3-4-5")

2025-01-14 17:09:07.747725: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-14 17:09:09.230162: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6119 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5
2025-01-14 17:09:09.842820: E tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:87] DefaultLogger 3: [runtime.cpp::~Runtime::346] Error Code 3: API Usage Error (Parameter check failed at: runtime/rt/runtime.cpp::~Runtime::346, condition: mEngineCounter.use_count() == 1. Destroying a runtime before destroying deserialized engines created by the runtime leads to undefined behavior.
)


In [2]:
from selfplay import play_game
from configs import selfplayConfig

_, _, _ = play_game(trt_func, tablebase, 2)



  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 8
  +---+---+---+---+---+---+---+---+
  |   |   |   | B |   |   |   |   | 7
  +---+---+---+---+---+---+---+---+
  |   | p |   | k |   |   |   |   | 6
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   | P | 5
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 4
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 3
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 2
  +---+---+---+---+---+---+---+---+
  |   |   |   | K |   |   |   |   | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: 8/3B4/1p1k4/7P/8/8/8/3K4 b - - 0 86
Key: c8c02598546d1e4b
Tablebase draw in 171 moves in 13.15 seconds


In [1]:
from rchess import Board
board = Board("r3r3/2P4k/3Bbbqp/ppQ2pp1/4pPP1/1P6/P1R2N1P/3R2K1 w - -")
print(board)
print(board.legal_moves())


  +---+---+---+---+---+---+---+---+
  | r |   |   |   | r |   |   |   | 8
  +---+---+---+---+---+---+---+---+
  |   |   | P |   |   |   |   | k | 7
  +---+---+---+---+---+---+---+---+
  |   |   |   | B | b | b | q | p | 6
  +---+---+---+---+---+---+---+---+
  | p | p | Q |   |   | p | p |   | 5
  +---+---+---+---+---+---+---+---+
  |   |   |   |   | p | P | P |   | 4
  +---+---+---+---+---+---+---+---+
  |   | P |   |   |   |   |   |   | 3
  +---+---+---+---+---+---+---+---+
  | P |   | R |   |   | N |   | P | 2
  +---+---+---+---+---+---+---+---+
  |   |   |   | R |   |   | K |   | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: r3r3/2P4k/3Bbbqp/ppQ2pp1/4pPP1/1P6/P1R2N1P/3R2K1 w - - 0 1
Key: 5a9e3985f34193a2
["g1f1", "g1h1", "g1g2", "f2e4", "f2h1", "f2d3", "f2h3", "c5a3", "c5e3", "c5b4", "c5d4", "c5b6", "c5a7", "d6e5", "d6e7", "d6f8", "d1a1", "d1b1", "d1c1", "d1e1", "d1f1", "d1d2", "d1d3", "d1d4", "d1d5", "c2c1", "c2b2", "c2d2", "c2e2", "c2c3", "c2c4", "

In [9]:
from datetime import datetime
datetime.now().strftime('%F_%T.%f')[:-3]

'2025-01-20_21:24:17.870'